# All imports necessary

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys

In [3]:
sys.path.append('..')

In [4]:
import os

In [5]:
import numpy as np

In [6]:
import pandas as pd

In [7]:
from source.code.utils.utils import filter_by_subcorpus
from source.code.utils.utils import get_tagged_texts_as_pd

from source.code.transformers.sentenceextractor import SentenceExtractor
from source.code.models.memorytagger import MemoryTagger
from source.code.models.bilstmtagger import BiLSTMTagger

from source.code.utils.preprocessing import filtrations
from source.code.utils.preprocessing import additional_features
from source.code.utils.preprocessing import crf_pre_processing

Using TensorFlow backend.


In [8]:
from sklearn_crfsuite.metrics import flat_classification_report as crfsuite_classification_report

from sklearn_crfsuite import metrics

from sklearn_crfsuite import CRF as sklearn_crf

In [9]:
from seqeval.metrics import classification_report as seqeval_classification_report

In [10]:
from seqeval.metrics import sequence_labeling as seqeval_classification_report

In [11]:
from sklearn.metrics import classification_report as sklearn_classification_report

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score

In [12]:
from keras.preprocessing.sequence import pad_sequences

In [13]:
import seaborn as sns

In [14]:
import matplotlib.pyplot as plt

In [15]:
sns.set(color_codes=True)

In [16]:
sns.set(font_scale=2)

In [17]:
%matplotlib inline

In [18]:
features = ['semantic_relation_tagged', 'animacy_tagged', 'lambda_dsr_len', 'word_sense_exists', 'is_title', 'contains_digits', 'word_len']
target = 'ner_tag'

In [19]:
pd.set_option('display.max_rows', 30000)

# Naive tag frequency memorization

In [ ]:
target_subcorpus_folders = filter_by_subcorpus('../data/datasets/gmb-2.2.0/', 'subcorpus: Voice of America')

In [ ]:
X = get_tagged_texts_as_pd(target_subcorpus_folders, '../data/datasets/gmb-2.2.0/')

In [ ]:
X.ner_tag.value_counts()

In [ ]:
X = filtrations(X)

In [ ]:
X.ner_tag.value_counts()

In [ ]:
X = additional_features(X)

In [ ]:
X.info()

In [ ]:
X, y = X.token, X.ner_tag

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

In [ ]:
estimator = MemoryTagger()

In [ ]:
scores = cross_val_score(estimator, X_train, y_train, cv=5, verbose=True, n_jobs=-1)

In [ ]:
scores

In [ ]:
estimator.fit(X_train, y_train)

In [ ]:
sklearn_report = sklearn_classification_report(y_pred=estimator.predict(X_test), y_true=y_test)
print(sklearn_report)

# Random Forest Classifier

In [ ]:
target_subcorpus_folders = filter_by_subcorpus('../data/datasets/gmb-2.2.0/', 'subcorpus: Voice of America')

In [ ]:
X = get_tagged_texts_as_pd(target_subcorpus_folders, '../data/datasets/gmb-2.2.0/')

In [ ]:
X.ner_tag.value_counts()

In [ ]:
X = filtrations(X)

In [ ]:
X.ner_tag.value_counts()

In [ ]:
X = additional_features(X)

In [ ]:
X.info()

In [ ]:
X, y = X[features], X[target]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

In [ ]:
estimator = RandomForestClassifier()

In [ ]:
scores = cross_val_score(estimator, X_train, y_train, cv=5, verbose=True, n_jobs=-1)

In [ ]:
scores

In [ ]:
estimator.fit(X_train, y_train)

In [ ]:
report = sklearn_classification_report(y_pred=estimator.predict(X_test), y_true=y_test)
print(report)

# HMM

[This article](https://pdfs.semanticscholar.org/9528/4b31f27b9b8901fdc18554603610ebbc2752.pdf)  gives a full description of how to calculate parameters for Hidden Markov Model.

From [this article](https://www.digitalvidya.com/blog/inroduction-to-hidden-markov-models-using-python/) we have taken Viterbi algorithm.

But first let's read the data again.

In [ ]:
target_subcorpus_folders = filter_by_subcorpus('../data/datasets/gmb-2.2.0/', 'subcorpus: Voice of America')

In [ ]:
X = get_tagged_texts_as_pd(target_subcorpus_folders, '../data/datasets/gmb-2.2.0/')

In [ ]:
X.head(30)

In [ ]:
X.ner_tag.value_counts()

Lots of redundant tag additions (like -nam, -dow).

We will work without them:

In [ ]:
X = filtrations(X, with_dots=True)

In [ ]:
X.head(30)

In [ ]:
X.ner_tag.value_counts()

In [ ]:
X.ner_tag.nunique()

The approach proposed in the article mentioned above is divided into three steps:
- Data preparation;
- Parameter estimation (or training);
- Usage of parameters estimated (or testing).

## Step 1: Data preparation

In [ ]:
X, y = SentenceExtractor().fit_transform(X)

In [ ]:
sentence_lenghts = list(map(len, y))

In [ ]:
max(sentence_lenghts)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Step 2: HMM Parameter Estimation

### Step 2.1: Find states.

In [ ]:
states = list(set([tag for sentence in y_train for tag in sentence]))
tag2idx = {t: i for i, t in enumerate(states)}
idx2tag = {i: w for w, i in tag2idx.items()}

### Step 2.2: Calculate Start probability (π)

In [ ]:
pi = np.zeros(len(states))

In [ ]:
for sentence in y_train:
    if len(sentence) > 0:
        pi[tag2idx[sentence[0]]] += 1

In [ ]:
pi = pi / len(y_train)

### Step 2.3: Calculate transition probability (A)

In [ ]:
A = np.zeros((len(states), len(states)))
A_i = np.zeros(len(states))

In [ ]:
for i in range(len(y_train)):
    if len(y_train[i]) > 1:
        for j in range(len(y_train[i]) - 1):
            A[tag2idx[y_train[i][j]]][tag2idx[y_train[i][j + 1]]] += 1
            A_i[tag2idx[y_train[i][j]]] += 1
        A_i[tag2idx[y_train[i][len(y_train[i]) - 1]]] += 1
for i in range(len(states)):
    A[i] = A[i] / A_i[i]

In [ ]:
plt.figure(figsize=(20, 12))
sns.heatmap(A, annot=True, xticklabels=states, yticklabels=states, fmt='.2g')
plt.tight_layout()
plt.show()

### Step 2.4: Calculate emission probability (B)

## Step 3: Testing

# CRF

In [ ]:
target_subcorpus_folders = filter_by_subcorpus('../data/datasets/gmb-2.2.0/', 'subcorpus: Voice of America')

In [ ]:
X = get_tagged_texts_as_pd(target_subcorpus_folders, '../data/datasets/gmb-2.2.0/')

In [ ]:
X.ner_tag.value_counts()

In [ ]:
X = filtrations(X, with_dots=True)

In [ ]:
X.ner_tag.value_counts()

In [ ]:
X = additional_features(X)

In [ ]:
X.info()

In [ ]:
X, y = crf_filtration_and_pre_processing(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
estimator = sklearn_crf(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=100, all_possible_transitions=False)

In [ ]:
scores = cross_val_score(estimator, X_train, y_train, cv=5, verbose=True, n_jobs=-1)

In [ ]:
scores

In [ ]:
estimator.fit(X_train, y_train)

In [ ]:
report = crfsuite_classification_report(y_pred=estimator.predict(X_test), y_true=y_test)
print(report)

# Bi-LSTM

In [ ]:
target_subcorpus_folders = filter_by_subcorpus('../data/datasets/gmb-2.2.0/', 'subcorpus: Voice of America')

In [ ]:
X = get_tagged_texts_as_pd(target_subcorpus_folders, '../data/datasets/gmb-2.2.0/')

In [ ]:
X.head(30)

In [ ]:
X.ner_tag.value_counts()

In [ ]:
X = filtrations(X, with_dots=True)

In [ ]:
X.ner_tag.value_counts()

In [ ]:
X.ner_tag.nunique()

In [ ]:
X, y = SentenceExtractor().fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
estimator = BiLSTMTagger(checkpoint_dir='../data/datasets/keras_model/')

In [ ]:
estimator.fit(X_train, y_train)

In [ ]:
y_pred = estimator.predict(X_test)

In [ ]:
y_test = [[estimator.tag2idx[w] for w in s] for s in y_test]
y_test = pad_sequences(maxlen=75, sequences=y_test, padding="post", value=estimator.tag2idx["O"])
y_test = [[estimator.idx2tag[w] for w in s] for s in y_test]

In [ ]:
crfsuite_report = crfsuite_classification_report(y_pred=y_pred, y_true=y_test)
print(crfsuite_report)

In [ ]:
seqeval_report = seqeval_classification_report(y_pred=y_pred, y_true=y_test)
print(seqeval_report)

# Conclusion